<a href="https://colab.research.google.com/github/weedge/doraemon-nb/blob/main/my_colab_gpu_topk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# task
【任务描述】
给定850万条规模的数据文件，每条数据是最大128维度的整型id向量 （称为doc），id取值范围是0-50000，给定一个最大128维的整型id向量（称为query），数据集可以扩散进行优化.

求query与doc全集内各数据交集个数平均分 topk (k=100); 这里定义item交集分数为：
query[i] == doc[j] (0<=i<query_size, 0<=j<doc_size) 算一个交集, 平均分为 query与doc交集数目/max(query_size,doc_size)
``` shell
./bin/query_doc_scoring <doc_file_name> <query_file_name> <output_filename>
```

# optimize
1. currency(cpu thread pool) + parallel(gpu warp pool): cpu(baseline) -> cpu thread currency -> cpu + gpu -> cpu thread currency + gpu
2. find or filter: use hash/bitmap(bloom)
3. topk sort: heap sort (partial_sort) -> bitonic sort (gpu parallel)
4. search: need build index (list(IVF,skip),tree or graph), orderly struct
5. SIMD: for cpu arch instruction set (sse,avx2,avx512)

# reference
- https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html
- https://docs.nvidia.com/cuda/cuda-c-best-practices-guide/index.html
- https://docs.nvidia.com/cuda/cuda-compiler-driver-nvcc/index.html
- https://docs.nvidia.com/cuda/cuda-runtime-api/index.html
- https://www.youtube.com/watch?v=cOBtkPsgkus






In [ ]:
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/nsight-systems-2023.2.3_2023.2.3.1001-1_amd64.deb
!apt update
!apt install ./nsight-systems-2023.2.3_2023.2.3.1001-1_amd64.deb
!apt --fix-broken install


In [ ]:
!wget "https://bj.bcebos.com/v1/ai-studio-online/9805dd2d2e8e472693efac637628e16b9f9c5be0fe30438bb4a80de3b386781a?responseContentDisposition=attachment%3B%20filename%3DSTI2_1017.zip&authorization=bce-auth-v1%2F5cfe9a5e1454405eb2a975c43eace6ec%2F2023-10-18T12%3A42%3A27Z%2F-1%2F%2F6b5388dcd9013bc9b340bb1806476afa938ce0c65f2f595e1a75f529e90e4187" -O STI2_1017.zip

In [ ]:
!rm -rf STI2 && unzip STI2_1017.zip && mv STI2\ 2 STI2

In [5]:
!sh STI2/build.sh

build success


### just dev/test with colab vim IDE ❤ 🐑 🐑 🐑

In [6]:
!mkdir -p querys

In [8]:
!touch querys/query0.txt

In [10]:
!rm -f docs.txt && touch docs.txt

In [21]:
!nvcc STI2/src/main.cpp STI2/src/topk.cu -o ./query_doc_scoring -I./ -ISTI2/src -L/usr/local/cuda/lib64 -lcudart -lcuda  -O3 -g

In [12]:
!./query_doc_scoring ./docs.txt ./querys ./res.txt

start get topk
query0.txt:10
query_size: 1
doc_size: 21
read file cost 0 ms 
end
query_idx:0 	 query_on_shm[query_idx]:10	 j:0	 doc_segment[j]:10	 tmp_score:1.0000000000 
query_idx:0 	 query_on_shm[query_idx]:10	 j:0	 doc_segment[j]:10	 tmp_score:1.0000000000 
query_idx:0 	 query_on_shm[query_idx]:10	 j:0	 doc_segment[j]:1	 tmp_score:0.0000000000 
query_idx:0 	 query_on_shm[query_idx]:10	 j:1	 doc_segment[j]:10	 tmp_score:2.0000000000 
query_idx:0 	 query_on_shm[query_idx]:10	 j:1	 doc_segment[j]:10	 tmp_score:1.0000000000 
query_idx:0 	 query_on_shm[query_idx]:10	 j:2	 doc_segment[j]:10	 tmp_score:3.0000000000 
query_idx:0 	 query_on_shm[query_idx]:10	 j:2	 doc_segment[j]:10	 tmp_score:2.0000000000 
query_idx:0 	 query_on_shm[query_idx]:10	 j:3	 doc_segment[j]:1	 tmp_score:2.0000000000 
query_idx:0 	 query_on_shm[query_idx]:10	 j:4	 doc_segment[j]:10	 tmp_score:3.0000000000 
doc_id:0	 scores[doc_id]:1.0000000000	 tmp_score:1.0000000000	 query_len:1	 doc_lens[doc_id]:1
doc_id:2	 scores

In [18]:
!nvcc -g -O3 -o ./threadpool_example threadpool_example.cpp -std=c++11

In [19]:
!./threadpool_example

hardware concurrency:2
result:0
result:1
cost 1004ms


In [13]:
!cat res.txt

368
0	2	4	1	3	5	6	7	8	9
